In [5]:
from PIL import Image
import matplotlib.pyplot as plt
import torch, os
from torchvision import transforms
from transformers import AutoModelForImageSegmentation

In [ ]:
model = AutoModelForImageSegmentation.from_pretrained('briaai/RMBG-2.0', trust_remote_code=True)
torch.set_float32_matmul_precision(['high', 'highest'][0])
model.to('cuda')
model.eval()

In [ ]:
# Data settings
image_size = (1024, 1024)
transform_image = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

image_paths = ['cola_baloon.png', 'restaurant_baloons.webp']  # замените на ваши пути к изображениям
images = [Image.open(os.path.join("input_imgs", path)).convert('RGB') for path in image_paths]
input_tensors = torch.stack([transform_image(img) for img in images]).to('cuda')

# Prediction
with torch.no_grad():
    preds = model(input_tensors)[-1].sigmoid().cpu()

for i, (img, pred) in enumerate(zip(images, preds)):
    pred_pil = transforms.ToPILImage()(pred.squeeze())
    mask = pred_pil.resize(img.size)
    img.putalpha(mask)
    img.save(os.path.join("output_imgs", f"processed_image_{i}.png"))

: 